#### Defining function to fetch weather data from API

In [0]:
@keep_log
def get_raw_weather(df=None):

    import requests
    from datetime import datetime
    from pyspark.sql.functions import col, udf
    from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType
    
    # weather data schema
    weather_schema = StructType([
        StructField('visibility', IntegerType(), True),
        StructField('timezone', IntegerType(), True),
        StructField('main', StructType([
            StructField('temp', FloatType(), True),
            StructField('feels_like', FloatType(), True),
            StructField('temp_min', FloatType(), True),
            StructField('temp_max', FloatType(), True),
            StructField('pressure', IntegerType(), True),
            StructField('humidity', IntegerType(), True),
            StructField('sea_level', IntegerType(), True),
            StructField('grnd_level', IntegerType(), True)
        ])),
        StructField('clouds', StructType([
            StructField('all', FloatType(), True)
        ])),
        StructField('sys', StructType([
            StructField('country', StringType(), True),
            StructField('sunrise', IntegerType(), True),
            StructField('sunset', IntegerType(), True)
        ])),
        StructField('dt', IntegerType(), True),
        StructField('coord', StructType([
            StructField('lon', FloatType(), True),
            StructField('lat', FloatType(), True)
        ])),
        StructField('name', StringType(), True),
        StructField('weather', ArrayType(StructType([
                    StructField('id', IntegerType(), True),
                    StructField('main', StringType(), True),
                    StructField('description', StringType(), True),
                    StructField('icon', StringType(), True)
        ]), True)),
        StructField('cod', IntegerType(), True),
        StructField('id', IntegerType(), True),
        StructField('wind', StructType([
            StructField('speed', IntegerType(), True),
            StructField('deg', IntegerType(), True),
            StructField('gust', FloatType(), True)
        ])),
        StructField('base', StringType(), True)
    ])

    API_key = '12d8aa00ad6c5c370e793d4f5db3f95e'

    # function to fetch from api
    def fetch_weather_data(id):
        url = f'https://api.openweathermap.org/data/2.5/weather?id={id}&appid={API_key}'
        result = requests.get(url)
        return result.json()
    fetch_weather_udf = udf(lambda x: fetch_weather_data(x), weather_schema)
    
    if df == None:
        df = spark.sql("select * from dim_city limit 5")

    df = df.withColumn('result', fetch_weather_udf(col('id'))).select('result')
        
    start = datetime.fromtimestamp(df.selectExpr("min(result.dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(result.dt)").first()[0])
    
    return df, start, end